# PROBLEM B5

Build and train a neural network model using the Daily Max Temperature.csv dataset.
Use MAE as the metrics of your neural network model.
We provided code for normalizing the data. Please do not change the code.
Do not use lambda layers in your model.

The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets

Desired MAE < 0.2 on the normalized dataset.

In [1]:
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [2]:
!pip install pandas==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [3]:
!pip install Pillow==10.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [4]:
!pip install scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [5]:
!pip install tensorflow==2.13.0

In [6]:
!pip install tensorflow-datasets==4.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.3
    Uninstalling tensorflow-datasets-4.9.3:
      Successfully uninstalled tensorflow-datasets-4.9.3


Imports

In [1]:
import numpy as np
import pandas as pd
import PIL as pil
import scipy as sc
import tensorflow as tf
import tensorflow_datasets as tfds

Check Version

In [2]:
print(np.__version__)
print(pd.__version__)
print(pil.__version__)
print(sc.__version__)
print(tf.__version__)
print(tfds.__version__)

1.24.3
2.0.3
10.0.0
1.10.1
2.13.0
4.9.2


Get the Data

In [3]:
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'

In [4]:
data = pd.read_csv(data_url)

In [5]:
data.head(5)

,Date,Temperature
0,1981-01-01,38.1
1,1981-01-02,32.4
2,1981-01-03,34.5
3,1981-01-04,20.7
4,1981-01-05,21.5


Input the Data into an array

In [6]:
time_data = data['Date']
temperature_data = data['Temperature']

In [7]:
time_data[0:5]

0    1981-01-01
1    1981-01-02
2    1981-01-03
3    1981-01-04
4    1981-01-05
Name: Date, dtype: object

In [8]:
temperature_data[0:5]

0    38.1
1    32.4
2    34.5
3    20.7
4    21.5
Name: Temperature, dtype: float64

In [9]:
time_data = np.array(time_data)
temperature_data = np.array(temperature_data)

In [10]:
temperature_data[0:5]

array([38.1, 32.4, 34.5, 20.7, 21.5])

Normalize the Data

In [11]:
# Normalization Function. DO NOT CHANGE THIS CODE
min=np.min(temperature_data)
max=np.max(temperature_data)
temperature_data -= min
temperature_data /= max

Split The Data

In [12]:
split = 2500
series_train = temperature_data[:split]
time_train = time_data[:split]

series_test = temperature_data[split:]
time_test = time_data[split:]

In [13]:
series_train.shape

(2500,)

Create a Windowed Dataset

In [14]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1) # makes the data into a [[],[],[]] array
  data = tf.data.Dataset.from_tensor_slices(series)
  data = data.window(window_size + 1, shift=1, drop_remainder = True)
  data = data.flat_map(lambda w: w.batch(window_size + 1))
  data = data.shuffle(shuffle_buffer)
  data = data.map(lambda w: (w[:-1], w[-1]))
  return data.batch(batch_size).prefetch(1)

In [15]:
window_size=64
batch_size=256
shuffle_buffer_size=1000

In [16]:
train_set = windowed_dataset(series_train, window_size, batch_size, shuffle_buffer_size)
test_set = windowed_dataset(series_test, window_size, batch_size, shuffle_buffer_size)

Model Creation

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(window_size, kernel_size = 1, input_shape=[None,1], activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [18]:
model.compile(loss='mae', optimizer='sgd', metrics=['mae'])

In [19]:
model.fit(train_set, epochs=200)

Epoch 1/200
10/10 [==============================] - 11s 65ms/step - loss: 0.2292 - mae: 0.2292
Epoch 2/200
10/10 [==============================] - 0s 28ms/step - loss: 0.1214 - mae: 0.1214
Epoch 3/200
10/10 [==============================] - 0s 25ms/step - loss: 0.1072 - mae: 0.1072
Epoch 4/200
10/10 [==============================] - 0s 28ms/step - loss: 0.1052 - mae: 0.1052
Epoch 5/200
10/10 [==============================] - 0s 27ms/step - loss: 0.1046 - mae: 0.1046
Epoch 6/200
10/10 [==============================] - 0s 26ms/step - loss: 0.1043 - mae: 0.1043
Epoch 7/200
10/10 [==============================] - 0s 26ms/step - loss: 0.1039 - mae: 0.1039
Epoch 8/200
10/10 [==============================] - 0s 25ms/step - loss: 0.1036 - mae: 0.1036
Epoch 9/200
10/10 [==============================] - 0s 27ms/step - loss: 0.1033 - mae: 0.1033
Epoch 10/200
10/10 [==============================] - 0s 29ms/step - loss: 0.1030 - mae: 0.1030
Epoch 11/200
10/10 [============================